### Data Description

In this project we will use a list of neighborhoods within the Austin area (via Wikipedia) to understand the general layout of the city. From there we will use a Geocoder package to find the geographical layout of the city and understand where populations of people are residing at. Finally the use of Foursquare will be used to find venue data for exsisting resturants in certain areas, we will find out if there are already exsisting italian or pizza resturants nearby that could compete with the location of the resturant. 